In [ ]:
!pip install ratsnlp

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
want_moel = "beomi/kcbert-base"
# koelectra : monologg/koelectra-base-v3-discriminator
# kcelectra : beomi/KcELECTRA-base
# 그 외 : https://huggingface.co/models 원하는 모델 찾아쓰면 됨 ~!~~

In [5]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name=want_model,
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=10,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [6]:
import os, csv
from ratsnlp.nlpbook.classification.corpus import ClassificationExample

In [30]:
def get_labels():
    return ["0","1"]

def num_labels():
  return len(get_labels())

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [8]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축
학습데이터를 만듭니다.

In [10]:
import os, csv

In [12]:
lines = list(csv.reader(open("train.txt", "r", encoding="utf-8"), delimiter="\t", quotechar='"'))
train_examples = []
for (i, line) in enumerate(lines):
    if i == 0:
        continue
    text_a, label = line
    train_examples.append(ClassificationExample(text_a=text_a, text_b=None, label=label))

In [13]:
from ratsnlp.nlpbook.classification import ClassificationDataset

In [14]:
from torch.utils.data.dataset import Dataset
from transformers import PreTrainedTokenizer
from ratsnlp.nlpbook.classification.arguments import ClassificationTrainArguments
from ratsnlp.nlpbook.classification.corpus import _convert_examples_to_classification_features


In [18]:
train_dataset = _convert_examples_to_classification_features(
                    train_examples,
                    tokenizer,
                    args,
                    label_list=get_labels(),
                )

In [19]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [20]:
lines = list(csv.reader(open("test.txt", "r", encoding="utf-8"), delimiter="\t", quotechar='"'))
test_examples = []
for (i, line) in enumerate(lines):
    if i == 0:
        continue
    text_a, label = line
    test_examples.append(ClassificationExample(text_a=text_a, text_b=None, label=label))

In [21]:
val_dataset = _convert_examples_to_classification_features(
                    test_examples,
                    tokenizer,
                    args,
                    label_list=get_labels(),
                )

from torch.utils.data import SequentialSampler

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화합니다.

In [32]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=num_labels(),
)

In [33]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 학습 준비
Task와 Trainer를 준비합니다.

In [34]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [35]:
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-doccls exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-doccls`)에 저장됩니다.

In [36]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [37]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name=want_moel,
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.28-v1.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [38]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [39]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [40]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "유용 (useful)" if torch.argmax(prob) == 1 else "비유용 (useless)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'useful_percent': f"유용 {positive_prob * 100}%",
        'useless_percent': f"비유용 {negative_prob * 100}%",
    }

In [41]:
inference_fn("너무 좋아요 저 건성인데 이거 쓰니까 괜찮았요")

{'negative_data': '비유용 0.99',
 'negative_width': '99.0%',
 'positive_data': '유용 0.01',
 'positive_width': '1.0%',
 'prediction': '비유용 (negative)',
 'sentence': '너무 좋아요 저 건성인데 이거 쓰니까 괜찮았어요'}

In [42]:
inference_fn("예전에 대용량으로 써보고 제 피부에 무난 너무 잘 맞아서 다 떨어질 때 쯤 다시 사야겠다! 라고 마음 먹은 토너예요! 근데 토너 본품 가격보다도 저렴한 가격에 앰플을 같이 끼워주길래 바로 구매했어요! 일단 리뷰를 시작하기 앞서 저는 수분을 채워주지 않으면 좁쌀이 올라오는 수분 부족 건성 피부를 갖고 있습니다! 그래서 수분을 채워주지 않으면 피부가 금방 상하고 거칠어지는데 요 제품은 여러번 레이어해서 발라주면 수분감은 너무 만족스러웠어요!  일단 성분이 너무 좋고 어성초가 여드름에 좋기로 유명한 성분인걸로 알고 있는데 제 피부에는 너무 잘 맞았어요! 잘못 바르면 여드름올라오는 예민한 피부인데 요건 아무 문제도 없었답니당! 냄새는 없는 편이에요! 무향이라고 샹각하시면 될 거 같아요! 그래서 향에 민감하신 분들도 잘 사용하실수 있을 것 같아요! 막 보습감이 엄청나진 않지만 두 세번 정도 챱챱 해주면 촉촉하고 다음 단계인 에센스를 더 잘 먹게 해주는 기분!! 완전 추천해요! 그리고 너무 기대됐던 앰플은 기대 이상이었어요! 본품 증정이더라구요…!! 여러분 안사고 뭐하세요…!! 할인까지 하고 있을 때 얼른 쟁여두세요!! 완전 혜자 구성…!!! 앰플은 토너 보다는 훠어얼씬 보습감과 유분감이 느껴졌어요! 그래서 토너 바르고 앰플 바르니까 너무너무 찰떡궁합이더라구요!! 앰플 다 쓰면 재구매하고 시퍼요ㅜㅜ 이런 기획 많이많이 내주세요!! 제가 다 살게요..!!")

{'negative_data': '비유용 0.0413',
 'negative_width': '4.130000000000001%',
 'positive_data': '유용 0.9587',
 'positive_width': '95.87%',
 'prediction': '유용 (positive)',
 'sentence': '예전에 대용량으로 써보고 제 피부에 무난 너무 잘 맞아서 다 떨어질 때 쯤 다시 사야겠다! 라고 마음 먹은 토너예요! 근데 토너 본품 가격보다도 저렴한 가격에 앰플을 같이 끼워주길래 바로 구매했어요! 일단 리뷰를 시작하기 앞서 저는 수분을 채워주지 않으면 좁쌀이 올라오는 수분 부족 건성 피부를 갖고 있습니다! 그래서 수분을 채워주지 않으면 피부가 금방 상하고 거칠어지는데 요 제품은 여러번 레이어해서 발라주면 수분감은 너무 만족스러웠어요!  일단 성분이 너무 좋고 어성초가 여드름에 좋기로 유명한 성분인걸로 알고 있는데 제 피부에는 너무 잘 맞았어요! 잘못 바르면 여드름올라오는 예민한 피부인데 요건 아무 문제도 없었답니당! 냄새는 없는 편이에요! 무향이라고 샹각하시면 될 거 같아요! 그래서 향에 민감하신 분들도 잘 사용하실수 있을 것 같아요! 막 보습감이 엄청나진 않지만 두 세번 정도 챱챱 해주면 촉촉하고 다음 단계인 에센스를 더 잘 먹게 해주는 기분!! 완전 추천해요! 그리고 너무 기대됐던 앰플은 기대 이상이었어요! 본품 증정이더라구요…!! 여러분 안사고 뭐하세요…!! 할인까지 하고 있을 때 얼른 쟁여두세요!! 완전 혜자 구성…!!! 앰플은 토너 보다는 훠어얼씬 보습감과 유분감이 느껴졌어요! 그래서 토너 바르고 앰플 바르니까 너무너무 찰떡궁합이더라구요!! 앰플 다 쓰면 재구매하고 시퍼요ㅜㅜ 이런 기획 많이많이 내주세요!! 제가 다 살게요..!!'}